In [ ]:
!pip install pandas beautifulsoup4 requests

In [ ]:
import os
import csv
import pandas as pd
from bs4 import BeautifulSoup
from requests import get


In [ ]:

def save_html_contents(link, page_name):
    '''
    Send request to destination link and -
    - save its content into file in local disk.
    '''
    resp = get(link)

    with open(f'/content/drive/MyDrive/scrape/pages/page_{page_name}.html', 'w',
              encoding='utf-8') as f:
        f.write(str(resp.text))


df = pd.read_csv('/content/drive/MyDrive/scrape/links.csv')
for index, row in df.iterrows():
  link = row['link']
  name = row['name']
  # File name can't have / char
  if '/' in name:
    name = name.split('/')[-1]
    name = name.replace('.html','')
  save_html_contents(link, name)

In [ ]:
# links['link'].head()

# A Sample for test

In [ ]:

html = None
with open(f'/content/drive/MyDrive/scrape/pages/page_16-bit computing - Wikipedia.html',
          'r', encoding='utf-8') as f:
    html = str(f.read())

soup = BeautifulSoup(html, 'html.parser')

paragraphs = soup.find_all('p')

for p in paragraphs:
    print(p.text)

In computer architecture, 16-bit integers, memory addresses, or other data units are those that are  16 bits (2 octets) wide.  Also, 16-bit central processing unit (CPU) and arithmetic logic unit (ALU) architectures are those that are based on registers, address buses, or data buses of that size. 16-bit microcomputers are microcomputers that use 16-bit microprocessors.

A 16-bit register can store 216 different values. The range of integer values that can be stored in 16 bits depends on the integer representation used. With the two most common representations, the range is 0 through 65,535 (216 − 1) for representation as an (unsigned) binary number, and −32,768 (−1 × 215) through 32,767 (215 − 1) for representation as two's complement. Since 216 is 65,536, a processor with 16-bit memory addresses can directly access 64 KB (65,536 bytes) of byte-addressable memory. If a system uses segmentation with 16-bit segment offsets, more can be accessed.

The MIT Whirlwind (c. 1951)[1][2] was qui

# Scrape from wiki

In [ ]:
def paragraphs_getter(file_name):
    '''
    Look at the htmls and find needed contents
    '''

    html = None
    with open(f'/content/drive/MyDrive/scrape/pages/{file_name}',
              'r', encoding='utf-8') as f:
        html = str(f.read())

    soup = BeautifulSoup(html, 'html.parser')

    paragraphs = soup.find_all('p')

    title = soup.find('h1', {'id': 'firstHeading'}).text
    print('_'*10,file_name,'|'*5,title)

    return paragraphs, title

directory = '/content/drive/MyDrive/scrape/pages'

# Get a list of files in the directory
file_list = os.listdir(directory)

paragraphs=[]
titles=[]


for file_name in file_list:
  p_list=[]
  content, title = paragraphs_getter(file_name)
  titles.append(title)

  for p in content:
    p_list.append(p.text)


  paragraphs.append(' '.join(p_list))




with open('articles.csv', 'w', newline='', encoding='utf-8') as file:
      writer = csv.writer(file)
      writer.writerow(["content","title"])


filename = 'articles.csv'
with open(filename, 'a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    for index, paragraph in enumerate(paragraphs):
        writer.writerow([paragraph,titles[index]])



__________ page_Public_health.html ||||| Public health
__________ page_Sexual health clinic - Wikipedia.html ||||| Sexual health clinic
__________ page_Smoking cessation - Wikipedia.html ||||| Smoking cessation
__________ page_African Championship - Wikipedia.html ||||| African Championship
__________ page_College Cup - Wikipedia.html ||||| College Cup
__________ page_European Championship - Wikipedia.html ||||| European Championship
__________ page_German Open - Wikipedia.html ||||| German Open
__________ page_Formula One World Champion - Wikipedia.html ||||| Formula One World Champion
__________ page_Defensive Player of the Year Award - Wikipedia.html ||||| Defensive Player of the Year Award
__________ page_Cooking - Wikipedia.html ||||| Cooking
__________ page_Freerice - Wikipedia.html ||||| Freerice
__________ page_Geographical indication - Wikipedia.html ||||| Geographical indication
__________ page_Jerk (cooking) - Wikipedia.html ||||| Jerk (cooking)
__________ page_Veganism.html

In [ ]:
pd.read_csv('articles.csv').head()


,content,title
0,"\n Public health is ""the science and art of pr...",Public health
1,Sexual health clinics specialize in the preven...,Sexual health clinic
2,"\n Smoking cessation, usually called quitting ...",Smoking cessation
3,An African Championship is a top level interna...,African Championship
4,College Cup may refer to:\n,College Cup


# Tokenizer and preproccesses

In [ ]:
import nltk
import re
from nltk import word_tokenize, WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

## clean the data
stopwordSet = set(stopwords.words("english"))

## NlP Processing
lemma = WordNetLemmatizer()

def cleanup_sentences(sentence):
    text = re.sub('[^a-zA-Z]'," ", sentence) # Removing non a-z characters
    text = text.lower() # Lowering all text
    text = word_tokenize(text, language="english") # Splitting each word into an element of a list
    text = [lemma.lemmatize(word) for word in text if(word) not in stopwordSet] # Lemmatizing words and removing stopwords
    text = " ".join(text) # Putting words back into a single string. ['the', 'brown', 'cow'] --> 'the brown cow'
    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Inverted Indexing

In [ ]:
def create_inverted_index(articles):
    inverted_index = {}

    # Iterate through each article
    for article_id, article_content in articles.items():
        # Tokenize the article into individual terms
        # terms = article_content.lower().split()
        terms = set(article_content.lower().split())  # Use a set to remove duplicates


        # Update the inverted index
        for term in terms:
            if term in inverted_index:
                # Add the article ID to the existing entry
                inverted_index[term].append(article_id)
            else:
                # Create a new entry with the article ID
                inverted_index[term] = [article_id]

    return inverted_index

directory = '/content/drive/MyDrive/scrape'
df = pd.read_csv(f'{directory}/articles.csv')#.head()
# Cleanup texts
df['content'] = df['content'].apply(cleanup_sentences)
df_dict = df.to_dict()
articles = df_dict['content']
titles = df_dict['title']
# print(titles)
# Change the key from old 0, 1, 2 ... to article title
for index, key in enumerate(list(articles)):
  new_key = titles[index]
  articles[new_key] = articles.pop(index)


inverted_index = create_inverted_index(articles)

index=0
# Print the inverted index
for term, document_ids in inverted_index.items():
    print(f"{term}: {document_ids}")

    index+=1
    if index > 10 :
      break


would: ['16-bit computing', 'HTTP 404', 'Ada (programming language)', 'Computer science', 'Computer security', 'Software', 'Random-access memory', 'Recursion (computer science)', 'Transport Layer Security', 'Service-oriented architecture', 'Unicode', 'Web browser', 'Windows Me', 'x86', 'Activism', 'Anti-capitalism', 'Anti-nuclear movement in the United States', 'Asian values', 'Consistent life ethic', 'Culture', 'Contempt of Parliament', 'Freedom of speech', 'Freedom of the press', 'Internet censorship', 'Islamism', 'Islamic state', 'Judiciary', 'Nazism', 'Political science', 'Sovereign state', 'Life expectancy', 'Assisted reproductive technology', 'Bacteria', 'Bipolar disorder', 'Famine', 'Disease', 'Health information on Wikipedia', 'Healthy diet', 'Nanotechnology', 'Public health', 'Smoking cessation', 'Cooking', 'Geographical indication', 'Jerk (cooking)', 'Veganism', 'Colotomy', 'Adaptive music', 'Battle', 'Iranian Revolution', 'Achaemenid Empire', 'Chi-squared test', 'Deletion (g

In [ ]:
# Save as CSV file using csv.writer()
with open(f'{directory}/inverted_index.csv', mode='w', newline='') as file:
	writer = csv.writer(file)
	for term, document_ids in inverted_index.items():
		writer.writerow([term] + document_ids)

# Bert

In [ ]:
# !pip install --upgrade joblib
!pip install bertopic sentence-transformers


In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from bertopic.representation import MaximalMarginalRelevance

In [ ]:
sentence_model = SentenceTransformer("all-distilroberta-v1")
topic_model = BERTopic(embedding_model=sentence_model, calculate_probabilities = True, verbose = True, )# diversity = 0.2


In [ ]:
### train the model on the documents
docs = list(articles.values())
topics, probabilities = topic_model.fit_transform(docs)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2023-07-02 07:35:18,511 - BERTopic - Transformed documents to Embeddings
2023-07-02 07:35:28,178 - BERTopic - Reduced dimensionality
2023-07-02 07:35:28,192 - BERTopic - Clustered reduced embeddings


In [ ]:
freq = topic_model.get_topic_info()
freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,20,-1_famine_cell_bacteria_food,"[famine, cell, bacteria, food, plasmid, life, ...",[cell basic structural functional unit life fo...
1,0,34,0_state_political_government_nazi,"[state, political, government, nazi, nuclear, ...",[political militant islam portal islamism also...
2,1,19,1_computer_bit_system_tl,"[computer, bit, system, tl, software, used, da...",[transport layer security tl cryptographic pro...
3,2,14,2_health_disorder_disease_bipolar,"[health, disorder, disease, bipolar, public, w...",[disease particular abnormal condition negativ...


In [ ]:
# topic_model.visualize_barchart()


In [ ]:
topic_model.get_topic_freq().head(20)


,Topic,Count
1,0,34
2,-1,20
0,1,19
3,2,14


In [ ]:
## topic model - visualize hierarchies
# topic_model.visualize_hierarchy(top_n_topics=40)

# Predict

In [ ]:
# Simulations a ARTICLE that we tagged it
similar_topics, similarity = topic_model.find_topics(articles['16-bit computing'], top_n = 3)

In [ ]:
topic_model.get_topic(similar_topics[0])

[('computer', 0.04178509404956643),
 ('bit', 0.03272534195966885),
 ('system', 0.031325203185991225),
 ('tl', 0.02751645387225997),
 ('software', 0.027112049755860103),
 ('used', 0.027070203193091773),
 ('data', 0.026746872149390453),
 ('character', 0.025891391999171693),
 ('memory', 0.02569398396907614),
 ('unicode', 0.02529093345572671)]

In [ ]:
# Simulations a QUERY that we tagged it
QUERY = 'Iran goverments'
similar_topics, similarity = topic_model.find_topics(QUERY, top_n = 3)

In [ ]:
topic_model.get_topic(similar_topics[0])

[('state', 0.02854877594862139),
 ('political', 0.023683445760862988),
 ('government', 0.01980291468975258),
 ('nazi', 0.018448050998873988),
 ('nuclear', 0.018360918887995216),
 ('also', 0.018009337585742316),
 ('power', 0.017954788964888998),
 ('freedom', 0.016313923081668416),
 ('islamic', 0.016240336312859194),
 ('right', 0.015788986682830266)]

# Find shared words

In [ ]:
l1=[('computer', 0.04178509404956643),
 ('bit', 0.03272534195966885),
 ('system', 0.031325203185991225),
 ('tl', 0.02751645387225997),
 ('software', 0.027112049755860103),
 ('used', 0.027070203193091773),
 ('data', 0.026746872149390453),
 ('character', 0.025891391999171693),
 ('memory', 0.02569398396907614),
 ('unicode', 0.02529093345572671)]
l2=[('computer', 0.04178509404956643), # proeccessors generation (Query)
 ('bit', 0.03272534195966885),
 ('system', 0.031325203185991225),
 ('tl', 0.02751645387225997),
 ('software', 0.027112049755860103),
 ('used', 0.027070203193091773),
 ('data', 0.026746872149390453),
 ('character', 0.025891391999171693),
 ('memory', 0.02569398396907614),
 ('unicode', 0.02529093345572671)]

print('SIMILARITY:',len(set(l1) & set(l2)),'/10')

len(l1), len(l2), len(set(l1)), len(set(l2))
# Completetly similar

SIMILARITY: 10 /10


(10, 10, 10, 10)

In [ ]:
l1=[('computer', 0.04178509404956643),
 ('bit', 0.03272534195966885),
 ('system', 0.031325203185991225),
 ('tl', 0.02751645387225997),
 ('software', 0.027112049755860103),
 ('used', 0.027070203193091773),
 ('data', 0.026746872149390453),
 ('character', 0.025891391999171693),
 ('memory', 0.02569398396907614),
 ('unicode', 0.02529093345572671)]
l2=[('state', 0.02854877594862139), # Iran goverments (Query)
 ('political', 0.023683445760862988),
 ('government', 0.01980291468975258),
 ('nazi', 0.018448050998873988),
 ('nuclear', 0.018360918887995216),
 ('also', 0.018009337585742316),
 ('power', 0.017954788964888998),
 ('freedom', 0.016313923081668416),
 ('islamic', 0.016240336312859194),
 ('right', 0.015788986682830266)]

print('SIMILARITY:',len(set(l1) & set(l2)),'/10')

len(l1), len(l2), len(set(l1)), len(set(l2))


SIMILARITY: 0 /10


(10, 10, 10, 10)

In [ ]:
for i in range(10):
    print(topic_model.get_topic(i))

[('state', 0.02854877594862139), ('political', 0.023683445760862988), ('government', 0.01980291468975258), ('nazi', 0.018448050998873988), ('nuclear', 0.018360918887995216), ('also', 0.018009337585742316), ('power', 0.017954788964888998), ('freedom', 0.016313923081668416), ('islamic', 0.016240336312859194), ('right', 0.015788986682830266)]
[('computer', 0.04178509404956643), ('bit', 0.03272534195966885), ('system', 0.031325203185991225), ('tl', 0.02751645387225997), ('software', 0.027112049755860103), ('used', 0.027070203193091773), ('data', 0.026746872149390453), ('character', 0.025891391999171693), ('memory', 0.02569398396907614), ('unicode', 0.02529093345572671)]
[('health', 0.0741411017980452), ('disorder', 0.051119543182297195), ('disease', 0.04637560296148824), ('bipolar', 0.04435719168651998), ('public', 0.036738133794732064), ('weight', 0.02866505578828898), ('smoking', 0.026863216592920697), ('surgery', 0.022621358581118297), ('may', 0.02261822536155333), ('loss', 0.0225959492

In [ ]:
# topic_model.visualize_heatmap()

In [ ]:
all_topics = topic_model.get_topics()

In [ ]:
freq.iloc[0]['Representation']
freq.iloc[0]['Representative_Docs']

# Save articles tags

In [ ]:
similarity_df = df.copy()

In [ ]:
# Insert similarity of each article to df
appended_list_tags=[]
for _, row in similarity_df.iterrows():
  similar_topics, similarity = topic_model.find_topics(row['content'], top_n = 3)
  appended_list_tags.append(topic_model.get_topic(similar_topics[0]))

similarity_df['tags'] = appended_list_tags

del appended_list_tags

similarity_df.head()

,content,title,tags
0,computer architecture bit integer memory addre...,16-bit computing,"[(computer, 0.04178509404956643), (bit, 0.0327..."
1,computer network communication http found erro...,HTTP 404,"[(computer, 0.04178509404956643), (bit, 0.0327..."
2,computer accessibility also known accessible c...,Computer accessibility,"[(computer, 0.04178509404956643), (bit, 0.0327..."
3,ada structured statically typed imperative obj...,Ada (programming language),"[(computer, 0.04178509404956643), (bit, 0.0327..."
4,asp net open source server side web applicatio...,ASP.NET,"[(computer, 0.04178509404956643), (bit, 0.0327..."


In [ ]:
similarity_df.to_csv(f'{directory}/articles_with_similarity.csv', index=False)

In [ ]:
QUERY = 'Iran goverment'
similar_topics, similarity = topic_model.find_topics(QUERY, top_n = 3)
tags_of_query = topic_model.get_topic(similar_topics[0])

Search into articles for most similar like query

In [ ]:
# print(set(tags_of_query),'\n\n')
results=[]
for _, row in similarity_df.iterrows():
  row['tags']
  # print('SIMILARITY:',len(set(tags_of_query) & set(row['tags'] )),'/10')
  # print(set(row['tags'] ))

  if len(set(tags_of_query) & set(row['tags'] )) > 6:
    results.append({row['title']:row['content']})

# Print search result

In [ ]:
for r in results:
  print(list(r.keys())[0])

# Print the name of finded articles:

A Scientific Support for Darwinism
Acclamation
Activism
Anti-capitalism
Anti-nuclear movement in the United States
Asian values
Center for American Politics and Citizenship
Consistent life ethic
Culture
Contempt of Parliament
Freedom of speech
Freedom of the press
Internet censorship
Island country
Islamism
Islamic state
Judiciary
May Conspiracy
Nazism
Neo-Gramscianism
Political science
Political statement
Tax Justice Network
Sovereign state
Jerk (cooking)
Veganism
Bayaty-Shiraz (mode)
Opera in English
Duet
Euouae
Music tourism
Battle
Iranian Revolution
Achaemenid Empire


# Save topic_model for later use in web or somewhere

In [ ]:
# import pickle
# from sentence_transformers import SentenceTransformer
# from bertopic import BERTopic


# with open(f'{directory}/topic_model.pkl', 'wb') as f:
#     pickle.dump(topic_model, f)

# # Save similarity_df for ease of use later
# with open(f'{directory}/similarity_df.pkl', 'wb') as f:
#     pickle.dump(similarity_df, f)

In [ ]:
import torch

# Save the BERTopic instance using torch.save
torch.save(topic_model, '/content/drive/MyDrive/scrape/topic_model_cpu.pth')